# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования**: выявить, какие критерии влияют на вероятность погашения займа в срок. \
**Задачи исследования**. Определить, существуют ли зависимости между погашением кредита в срок и:
- наличием детей у заемщика, 
- его семейного положения, 
- уровня его дохода, 
- целью кредита.

## Знакомство с данными

In [5]:
#импортируем библиотеки, которые понадобятся нам в исследовании

import pandas as pd  
from pymystem3 import Mystem
from collections import Counter
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian') 

In [6]:
initial_data = pd.read_csv('/datasets/data.csv') #применяем специальный метод для чтения файла формата csv
initial_data.info() #запрашиваем и выводим общую информацию об исходном датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [7]:
#произведем обзор полученных данных, вызвав 10 верхних и нижних строк датафрейма

display(initial_data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [8]:
display(initial_data.tail(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [9]:
#вызовем метод describe() для изучения основных статистических характеристик полученных данных
initial_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Всего в датафрейме 12 столбцов и 21525 строк. Знакомство с данными позволило выявить следующие проблемы, требующие решения:
- в столбцах days_employed и total_income присутствуют пропуски значений
- в столбцах children и days_employed присутствуют отрицательные значения, которых там быть не должно
- в столбце education присутствуют дубликаты, связанные с регистром букв
- в датафрейме присутствуют данные, которые, с большой долей вероятности, не понадобятся в исследовании: сведения об образовании, а также идентификаторы образования и семейного положения
- минимальный возраст клиента равен 0
- некорректные, малоинформативные и сбивающие с толку названия некоторых столбцов
- непонятна единица измерения в столбце total_income
- слишком большое количество знаков после запятой в данных total_income может затруднить анализ
- неверный тип данных в столбце dob_years

## Предобработка данных

### Обработка пропусков

Начнем предобработку данных с переименования столбцов. Дадим новые, более конкретные и при этом информативные названия столбцам children, dob_years, income_type, purpose и total_income, сохранив стиль написания.                                                  

In [10]:
#используем метод rename()
initial_data = initial_data.rename(columns={'children':'number_of_children', 'dob_years':'age_years', 'income_type':'type_of_employment', 'total_income':'monthly_income', 'purpose':'purpose_of_credit'})

#проверим результат
print(initial_data.columns)

Index(['number_of_children', 'days_employed', 'age_years', 'education',
       'education_id', 'family_status', 'family_status_id', 'gender',
       'type_of_employment', 'debt', 'monthly_income', 'purpose_of_credit'],
      dtype='object')


Столбцы переименованы, теперь их названия несут больше полезной информации. Далее займемся пропусками и для начала посчитаем их количество в каждом столбце таблицы.

In [11]:
print(initial_data.isna().sum())

number_of_children       0
days_employed         2174
age_years                0
education                0
education_id             0
family_status            0
family_status_id         0
gender                   0
type_of_employment       0
debt                     0
monthly_income        2174
purpose_of_credit        0
dtype: int64


Как уже было выявлено на этапе знакомства с данными, пропуски есть в столбцах days_employed и monthly_income, причем количество пропусков в них совпадает. На мой взгляд, возможны две причины появления данных пропусков: ошибка при выгрузке данных, либо данные клиенты никогда не работали в силу определенных обстоятельств (возраст, состояние здоровья, финансовое положение семьи и тд). Тогда можно заменить данные пропуски на нули, так как постановка средних или медианных значений может исказить данные и итоговые выводы.

In [12]:
initial_data = initial_data.fillna(0) #заменяем пропущенные значения на 0    

In [13]:
print(initial_data.isna().sum()) #проверяем, не остались ли пропуски в таблице

number_of_children    0
days_employed         0
age_years             0
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
type_of_employment    0
debt                  0
monthly_income        0
purpose_of_credit     0
dtype: int64


Мы заменили имеющиеся пропуски на 0, а не удалили соответствующие строки, так как помимо данных о стаже и доходе они содержат сведения о семейном положении, количестве детей клиентов и целях кредита, взаимосвязь с которыми нам также нужно выяснить. Поэтому на данном шаге был применен метод fillna(), а не dropna().

От пропусков в данных избавились, теперь откорректируем тип данных в столбце age_years.

### Замена типа данных

In [14]:
initial_data['age_years'] = initial_data['age_years'].astype(int) #заменим тип данных в столбце на int64
print(initial_data['age_years']) #проверим результат

0        42
1        36
2        33
3        32
4        53
         ..
21520    43
21521    67
21522    38
21523    38
21524    40
Name: age_years, Length: 21525, dtype: int64


Одной из выявленных на первом этапе проблем стало большое количество знаков после запятой в данных столбца total_income.  Применим к ним функцию round, чтобы ограничить количество знаков до двух:

In [15]:
initial_data['monthly_income'] = round(initial_data['monthly_income'], 2)
print(initial_data['monthly_income'])

0        253875.64
1        112080.01
2        145885.95
3        267628.55
4        158616.08
           ...    
21520    224791.86
21521    155999.81
21522     89672.56
21523    244093.05
21524     82047.42
Name: monthly_income, Length: 21525, dtype: float64


**Вывод:** Возраст относится к количественным данным, и в дальнейшем нам придется сравнивать клиентов по этому признаку и, возможно, объединять в категории. Удобнее это делать, когда возраст указан в виде полных лет, без учета месяцев. Поэтому на данном шаге мы изменили формат данных в столбце age_years на целочисленные.
К столбцу monthly_income мы применили функцию, позволившую округлить данные до двух знаков после запятой. Это также облегчит работу с ними в дальнейшем.

Обзор данных позволил также выявить наличие в них дубликатов, которые необходимо удалить. Этому будет посвящен следующий шаг исследования.

### Обработка дубликатов

Для начала посчитаем количество явных дубликатов в таблице:

In [16]:
print(initial_data.duplicated().sum())

54


Обзор данных позволил выявить проблемы с регистром букв в столбце education. Это может мешать выявлению и подсчету явных дубликатов, поэтому нужно привести данные к единому регистру. Используем метод str.lower():

In [17]:
initial_data['education'] = initial_data['education'].str.lower() #приводим все буквы к нижнему регистру
print(initial_data.duplicated().sum()) #вновь считаем дубликаты

71


Как мы можем видеть, после корректировки регистра количество выявленных дубликатов действительно увеличилось. Удалим дубликаты и откорректируем индексацию:

In [18]:
initial_data = initial_data.drop_duplicates().reset_index(drop=True) #удаление дублей
print(initial_data.duplicated().sum()) #проверка

0


**Вывод:** Явные дубликаты устранены. Возможная причина их возникновения: один клиент мог неоднократно брать кредит в банке, и каждый раз его данные вносились в базу заново, а проверка на то, что данные этого клиента в ней уже есть, не проводилась.

Теперь разберемся с отрицательными значениями в столбцах number_of_children и days_employed. Для начала убедимся, что в столбце с данными о количестве детей присутствует только одно отрицательное значение: - 1. Используем для этого метод unique():

In [19]:
print(initial_data['number_of_children'].unique())

[ 1  0  3  2 -1  4 20  5]


Отрицательное значение действительно одно, однако в столбце присутствует другое значение, которое выбивается из общего ряда и выглядит недостоверным: 20. Проверим, как часто встречается это значение:

In [20]:
print(initial_data[initial_data['number_of_children'] == 20]['number_of_children'].count())

76


76 клиентов с 20 детьми выглядит малоправдоподобно. Проверим, какой у них возраст:

In [21]:
print(initial_data[initial_data['number_of_children'] == 20].groupby('age_years')['age_years'].count())

age_years
0     1
21    1
23    1
24    1
25    1
26    1
27    2
29    2
30    3
31    2
32    2
33    2
34    3
35    2
36    2
37    4
38    1
39    1
40    4
41    2
42    3
43    2
44    2
45    3
46    3
48    1
49    3
50    3
51    1
52    1
53    1
54    1
55    1
56    5
57    1
59    2
60    1
61    1
62    1
64    1
69    1
Name: age_years, dtype: int64


В выборку попало большое количество клиентов, младше 40 и даже 30 лет, поэтому мужно сделать вывод, что значение 20 попало в данные по ошибке. Вычислим долю клиентов с некорректно указанным количеством детей: 

In [25]:
share_of_error_children = 76/21525 #вычислим долю клиентов с 20-ю детьми
share_of_error_child = (initial_data[initial_data['number_of_children'] == -1]['number_of_children'].count())/21525 #вычислим долю клиентов с -1 ребенком
print('Доля клиентов с 20-ю детьми: {:.2%}'.format(share_of_error_children))
print('Доля клиентов с минус 1 ребенком: {:.2%}'.format(share_of_error_child)) 

Доля клиентов с 20-ю детьми: 0.35%
Доля клиентов с минус 1 ребенком: 0.22%


Совокупная доля клиентов с аномальным количеством детей в датасете менее 1 %. Заменим некорректное значение 20 на 2, а - 1 на 1 и вновь проверим таблицу на наличие дубликатов.

In [230]:
initial_data['number_of_children'] = initial_data['number_of_children'].replace(20,2) #меняем ошибочное значение
initial_data['number_of_children'] = initial_data['number_of_children'].replace(-1,1) #меняем отрицательное значение
print(initial_data['number_of_children'].unique()) #проверяем уникальные значения в столбце
print(initial_data.duplicated().sum()) #вновь считаем явные дубликаты                                                               

[1 0 3 2 4 5]
0


Изменения некорректных значений в столбце number_of_children не повлияли на количество дубликатов, поэтому можно перейти к обработке отрицательных значений стажа. Перед этим вновь оставим напоминание о том, что в столбце возраста присутствует значение 0, появление которого тоже нужно будет исследовать.

In [231]:
initial_data['days_employed'] = initial_data['days_employed'].abs() #избавляемся от отрицательных значений
print(initial_data['days_employed'].describe()) #просмотрим статистические характеристики столбца на предмет других аномалий

count     21454.000000
mean      60355.501029
std      133531.435468
min           0.000000
25%         622.434168
50%        1821.731499
75%        4797.503489
max      401755.400475
Name: days_employed, dtype: float64


Вызов метода describe() позволил выявить две проблемы: работать с данными о трудовом стаже, выраженном в днях, не очень удобно, а максимальный стаж перевалил за 1000 лет, что явно указывает на ошибку. Для начала добавим в исходную таблицу новый столбец со стажем, выраженным в годах:

In [232]:
initial_data['years_employed'] = initial_data['days_employed'] / 365
print(initial_data['years_employed'].describe())

count    21454.000000
mean       165.357537
std        365.839549
min          0.000000
25%          1.705299
50%          4.991045
75%         13.143845
max       1100.699727
Name: years_employed, dtype: float64


In [233]:
initial_data['years_employed'] = initial_data['years_employed'].astype(int) #приведем значение к целочисленному формату для удобства
initial_data['years_employed'].sort_values(ascending=True) #отсортируем данные столбца по возрастанию
print(initial_data['years_employed'].median()) #найдем медианное значение

4.0


В среднем рабочий стаж клиентов банка составляет около 4 лет, при этом есть те, кто не работал (0) и некорректно большие значения (1100) (из-за них нельзя было использовать метод mean(), так как его результат отразил бы искаженную ситуацию). Что делать с последними, пока непонятно, но возможно, дальнейший анализ прояснит ситуацию.
Перейдем к возрасту клиентов. Подсчитаем, сколько клиентов того или иного возраста присутствует в таблице:

In [234]:
print(initial_data.groupby('age_years')['age_years'].count())

age_years
0     101
19     14
20     51
21    111
22    183
23    252
24    264
25    357
26    408
27    493
28    503
29    544
30    537
31    559
32    509
33    581
34    601
35    616
36    554
37    536
38    597
39    572
40    607
41    605
42    596
43    512
44    545
45    496
46    472
47    477
48    536
49    508
50    513
51    446
52    484
53    459
54    476
55    443
56    483
57    456
58    454
59    443
60    374
61    354
62    348
63    269
64    260
65    193
66    182
67    167
68     99
69     85
70     65
71     56
72     33
73      8
74      6
75      1
Name: age_years, dtype: int64


In [235]:
share_of_error = 101/21525 #вычислим долю клиентов с возрастом 0 лет - нулевым возрастом
print('Доля клиентов с нулевым возрастом: {:.2%}'.format(share_of_error)) 

Доля клиентов с нулевым возрастом: 0.47%


Менее 1 %. Казалось бы, этими данными можно пренебречь и удалить строки клиентов с нулевым возрастом. Однако возраст клиентов не является ключевой для целей данного исследования характеристикой, среди задач не числится выявление его взаимосвязи с фактом возврата долга в срок. Поэтому строки с нулевым возрастом лучше будет оставить и использовать данные из других столбцов для формирования окончательных выводов. \
То же самое можно отнести и к данным о трудовом стаже клиентов и некорректных значениях в соответствующем столбце.

Посмотрим, какие данные встречаются в оставшихся столбцах: family_status, gender и type_of_employment.

In [236]:
print(initial_data['family_status'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


In [237]:
print(initial_data['gender'].unique())

['F' 'M' 'XNA']


In [238]:
print(initial_data['type_of_employment'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


Особых проблем в этих столбцах нет. В семейном положении есть нарушение регистра, но оно не повлияло на данные. В столбце гендера есть указание на Х-гендер - XNA. В типах занятости все в порядке. 
Исправим регистр статуса "не женат":

In [239]:
initial_data['family_status'] = initial_data['family_status'].str.lower() #приводим все данные к ниднему регистру
print(initial_data['family_status'].unique()) #осуществляем проверку

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


### Лемматизация

Перейдем к целям кредита и для начала посмотрим, на что и как часто клиенты банка берут кредит:

In [240]:
print(initial_data.groupby('purpose_of_credit')['purpose_of_credit'].count())

purpose_of_credit
автомобили                                478
автомобиль                                494
высшее образование                        452
дополнительное образование                460
жилье                                     646
заняться высшим образованием              496
заняться образованием                     408
на покупку автомобиля                     471
на покупку подержанного автомобиля        478
на покупку своего автомобиля              505
на проведение свадьбы                     768
недвижимость                              633
образование                               447
операции с жильем                         652
операции с коммерческой недвижимостью     650
операции с недвижимостью                  675
операции со своей недвижимостью           627
покупка жилой недвижимости                606
покупка жилья                             646
покупка жилья для сдачи                   651
покупка жилья для семьи                   638
покупка коммерче

Даже на первый взгляд видно, что некоторые слова повторяются при указании целей в различных вариациях. Чтобы выявить эти слова, применим лемматизацию:

In [241]:
m = Mystem() 
unique_purposes_of_credit = initial_data['purpose_of_credit'].unique() #создадим список уникальных целей
string = ' '.join(unique_purposes_of_credit)
lemmas = m.lemmatize(string)
print(Counter(lemmas)) 

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


Теперь можно выделить основные цели, на которые клиенты банка брали кредиты.

Основными целями стали:
- недвижимость;
- автомобиль;
- образование;
- свадьба.

### Категоризация данных

Объединим ключевые для решения поставленных задач характеристики клиентов банка в укрупненные группы-категории. \
По количеству детей разделим клиентов на бездетных, многодетных и тех, кто имеет детей, но не попадает в категорию многодетных. \
По семейному положению разделим их на тех, кто имеет партнера, и тех, кто одинок. \
По ежемесячному доходу клиентов можно разделить на имеющих низкий, высокий или средний уровень дохода, а также тех, кто получает сверхдоход.\
Используем для этого функции с вложенным циклом if. 

Разобьем заемщиков банка на категории по количеству детей:

In [242]:
def having_children(number_of_children): # функция для категоризации клиентов по количеству детей
    if number_of_children==0:
        return 'бездетный'
    if 1<=number_of_children<=2:
        return 'есть дети'
    if number_of_children>=3:
        return 'многодетный'
    
initial_data['having_children'] = initial_data['number_of_children'].apply(having_children) # добавим в таблицу с исходными данными
                                                                                            # столбец с результатом применения функции
print(initial_data['having_children'].head(10))   # вызовем первые строки нового столбца для проверки работы функции     

0      есть дети
1      есть дети
2      бездетный
3    многодетный
4      бездетный
5      бездетный
6      бездетный
7      бездетный
8      есть дети
9      бездетный
Name: having_children, dtype: object


Повторим тот же порядок действий для категоризации по семейному положению и уровню дохода.

In [243]:
def family (family_status): # функция для категоризации клиентов по семейному положению
    if family_status=='женат / замужем' or family_status=='гражданский брак':
        return 'семейный'
    if family_status=='вдовец / вдова' or family_status=='в разводе' or family_status=='не женат / не замужем':
        return 'одинокий'
    
initial_data['family'] = initial_data['family_status'].apply(family)

print(initial_data['family'].tail(10)) # выводим на экран последние строки, так как они показывают работу функции 
                                       # чуть более наглядно, чем первые

21444    семейный
21445    семейный
21446    семейный
21447    семейный
21448    одинокий
21449    семейный
21450    семейный
21451    семейный
21452    семейный
21453    семейный
Name: family, dtype: object


Для того, чтобы разделить клиентов по уровню дохода, полезно будет сначала разбить все имеющиеся данные о доходах на равные группы. В этом нам помогут квартили. с помощью функции quantile() разделим доходы на четыре равные группы, а затем используем полученные результат для категоризации заемщиков.

In [244]:
print(initial_data['monthly_income'].quantile([0.25, 0.5, 0.75])) #ищем квантили

0.25     89089.0425
0.50    135781.0400
0.75    195813.9850
Name: monthly_income, dtype: float64


In [245]:
def income_level(monthly_income): # функция для категоризации клиентов по уровню дохода
    if monthly_income <= 89089:
            return 'Низкий уровень дохода'
    if monthly_income <= 135781:
            return 'Средний уровень дохода'
    if monthly_income <= 195814:
            return 'Высокий уровень дохода'
    if monthly_income > 195814:
            return 'Очень высокий уровень дохода'
        
initial_data['income_level'] = initial_data['monthly_income'].apply(income_level)

print(initial_data['income_level'].head(10))

0    Очень высокий уровень дохода
1          Средний уровень дохода
2          Высокий уровень дохода
3    Очень высокий уровень дохода
4          Высокий уровень дохода
5    Очень высокий уровень дохода
6    Очень высокий уровень дохода
7          Высокий уровень дохода
8          Средний уровень дохода
9          Высокий уровень дохода
Name: income_level, dtype: object


Для категоризации клиентов по целям кредита использованный ранее подход не подойдет из-за большой вариантивности данных в текстовом формате. Однако мы можем использовать полученные ранее результаты лемматизации. Создадим вручную список, в который включим выявленные цели, а также добавим к нему элемент "жилье" как вариант "недвижимости", и используем стемминг.  

In [246]:
words = ['недвижимость', 'автомобиль', 'образование', 'свадьба', 'жилье']
for word in words:
    print ('Исходное слово - ' + word + ', после стемминга - ' + russian_stemmer.stem(word))

Исходное слово - недвижимость, после стемминга - недвижим
Исходное слово - автомобиль, после стемминга - автомобил
Исходное слово - образование, после стемминга - образован
Исходное слово - свадьба, после стемминга - свадьб
Исходное слово - жилье, после стемминга - жил


Теперь мы можем создать новую функцию для категоризации клиентов банка по целям, для которых они взяли кредит:

In [247]:
def purposes_categories(purpose_of_credit): # функция для категоризации клиентов по уровню дохода
    if 'недвижим' in purpose_of_credit or 'жил' in purpose_of_credit:
            return 'Недвижимость'
    if 'автомобил' in purpose_of_credit:
            return 'Автомобиль'
    if 'образован' in purpose_of_credit:
            return 'Образование'
    if 'свадьб' in purpose_of_credit:
            return 'Свадьба'
        
initial_data['purposes_categories'] = initial_data['purpose_of_credit'].apply(purposes_categories)

print(initial_data['purposes_categories'].unique()) # проверка работы функции

['Недвижимость' 'Автомобиль' 'Образование' 'Свадьба']


**Вывод:** С помощью созданных функций и нескольких дополнительных методов мы разделили клиентов на категории по таким ключевым характеристикам, как: количество детей, семейное положение, сумма дохода в месяц и цели кредита. В дальнейшем это облегчит поиск взаимосвязей указанных характеристик с фактом погашения кредита клиентом в срок. Следующий шаг исследования будет посвящен именно этому.

## Проверка наличия зависимостей критериев и ответы на вопросы

Переходим к третьему этапу исследования, на котором мы будем искать ответы на вопросы, чтобы на их основе сформулировать итоговые выводы.\
Начнем с вопроса:
- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [248]:
children_pivot = initial_data.pivot_table(index = ['having_children','number_of_children'], \
                                columns = ['debt'], \
                                values = 'gender', aggfunc='count').reset_index()

children_pivot['ratio'] = children_pivot[1] / children_pivot[0] * 100
display (children_pivot)

debt,having_children,number_of_children,0,1,ratio
0,бездетный,0,13028.0,1063.0,8.159349
1,есть дети,1,4410.0,445.0,10.090703
2,есть дети,2,1926.0,202.0,10.488058
3,многодетный,3,303.0,27.0,8.910891
4,многодетный,4,37.0,4.0,10.810811
5,многодетный,5,9.0,NaN,NaN


**Вывод:** Исходя из данных таблицы, клиенты, имеющие детей, реже отдают кредиты в срок. При этом нужно отметить два важных момента:
- данные по клиентам с 5 детьми не показательны, так как мы не имеем сведений о тех заемщиках данной категории, которые имели задолженности по кредиту;
- клиенты с 3 детьми погашают кредиты в срок заметно чаще, чем их соседи по категории и те, у кого есть дети. Однако у нас недостаточно данных, чтобы выдвинуть гипотезу о том, с чем это связано. 

В целом же видна тенденция к тому, что чем больше у клиента детей, тем больше вероятность, что он просрочит платеж по кредиту.

Теперь исследуем взаимосвязь между возвратом кредита в срок и семейным положением заемщика и ответим на вопрос:.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [249]:
famyly_pivot = initial_data.pivot_table(index = ['family','family_status'], \
                                columns = ['debt'], \
                                values = 'gender', aggfunc='count').reset_index()

famyly_pivot['ratio'] = famyly_pivot[1] / famyly_pivot[0] * 100
display (famyly_pivot)

debt,family,family_status,0,1,ratio
0,одинокий,в разводе,1110,85,7.657658
1,одинокий,вдовец / вдова,896,63,7.031250
2,одинокий,не женат / не замужем,2536,274,10.804416
3,семейный,гражданский брак,3763,388,10.310922
4,семейный,женат / замужем,11408,931,8.160940


**Вывод:** Ознакомившись с данными таблицы, можно заметить, что клиенты, состоящие или состоявшие в законном браке, погашают кредиты в срок чаще, чем те, кто никогда не узаконивал отношения. Можно предположить, что законный брак развивает в людях ответственность - в частности за супруга и семью,- которая остается с ними даже после прекращения брачных отношений по тем или иным причинам. Эта ответственность и побуждает клиентов погашать кредиты в срок. В отличие от оформленного, в гражданском браке супруги больше рассчитывают на себя, поэтому такая повышенная ответственность не формируется. Отсюда и более высокая доля должников.

Теперь ответим на вопрос

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [250]:
income_level_pivot = initial_data.pivot_table(index = ['income_level'], \
                                columns = ['debt'], \
                                values = 'gender', aggfunc='count').reset_index()

income_level_pivot['ratio'] = income_level_pivot[1] / income_level_pivot[0] * 100
display (income_level_pivot)

debt,income_level,0,1,ratio
0,Высокий уровень дохода,4882,481,9.852519
1,Низкий уровень дохода,4945,419,8.473205
2,Очень высокий уровень дохода,4981,383,7.689219
3,Средний уровень дохода,4905,458,9.337411


**Вывод:** В результатах данного этапа исследования не прослеживается ни одной закономерности, поэтому можно сделать вывод, что зависимость между уровнем дохода и возвратом кредита в срок отсутствует.

Осталось изучить взаимосвязь своевременного погашения кредита и целей, для которых он был взят. Для этого ответим на вопрос

- Как разные цели кредита влияют на его возврат в срок?

In [251]:
purpose_pivot = initial_data.pivot_table(index = ['purposes_categories'], \
                                columns = ['debt'], \
                                values = 'gender', aggfunc='count').reset_index()

purpose_pivot['ratio'] = purpose_pivot[1] / purpose_pivot[0] * 100
display (purpose_pivot)

debt,purposes_categories,0,1,ratio
0,Автомобиль,3903,403,10.325391
1,Недвижимость,10029,782,7.797388
2,Образование,3643,370,10.156464
3,Свадьба,2138,186,8.699719


**Вывод:** Как показывает таблица, лучше всего возвращаются кредиты, взятые на операции с недвижимостью и свадьбой. Это выглядит вполне логично. Риск недоучиться либо лишиться автомобиля выглядит не таким серьезным по сравнению с риском остаться без крыши над головой. Что касается кредитов на свадьбу, то их суммы, вероятно, ниже, чем по остальным категориям, а отдавать, скорее всего, такой кредит будут с доходов обоих супругов, что также снижает риск возникновения задолженности. 

## Итоговые выводы и рекомендации

Целью проведенного иследования был поиск зависимостей между такими характеристиками клиентов как количество детей, семейное положение и уровень дохода; целью взятия кредита и погашением кредита в срок.
Результаты показали следующее:
- существует обратная зависимость погашения кредита в срок от количества детей клиента: люди с бОльшим количеством детей чаще имели задолженность. При этом существует аномалия клиентов с 3 детьми, которая не нашла объяснения;
- зависимость от семейного положения по типу "семейный-одинокий" найдена не была, однако было выявлено, что клиенты находящиеся или находившиеся в законном браке чаще отдают кредиты в срок;
- зависимости между уровнем дохода и возвратом кредита в срок не обнаружено;
- факт погашения кредита в срок зависит от целей его взятия: чаще погашаются кредиты, выданные на операции с недвижимостью и свадьбу, реже - на образование и автомобили.

На основе проведенного анализа можно сформулировать следующие рекомендации для банка:
- ввести систему проверки наличия данных в базе с целью выявления повторных клиентов и предупреждения появления дубликатов (возможно, стоит продумать систему присвоения каждому клиенту уникального идентификатора);
- производить подсчет трудового стажа в годах, а не днях: это удобнее как для дальнейшего анализа, так и для самих клиентов, так как большинство не знает свой стаж, выраженный в днях;
- разработать систему блокировок внесения данных из заявок с заведомо некорректными (отрицательными, слишком большими и тд) значениями;
- изменить способ подсчета дохода клиентов, чтобы результат выводился целым числом, либо с двумя знаками после запятой;
- стандартизировать варианты ответов на вопрос об образовании клиентов с целью избежания появления дубликатов, связанных с регистром букв.
